### Hito3 para Rendimiento Escolar
*"Modelación descriptiva, que busca definir cuáles son los principales determinantes del objeto de estudio. En base a esta sección se podrá construir o depurar el modelo predictivo."*

####  Carga de librerias e importación de .csv obtenido en el hito anterior

In [3]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula as sm
import statsmodels.formula.api as smf

#from graficos import *
import funciones as fun

import warnings
warnings.filterwarnings('ignore')

#plt.style.use('seaborn') # Gráficos estilo seaborn
#plt.rcParams["figure.figsize"] = (8,6) # Tamaño gráficos (5, 3)
#plt.rcParams["figure.dpi"] = 75 # resolución gráficos 100

In [4]:
#Se realiza la importación de los datos y ajuste del separador
df = pd.read_csv('students_ready.csv', index_col=0)
df

,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
school,,,,,,,,,,,,,,,,,,,,,
GP,F,NaN,U,GT3,A,4.0,4.0,at_home,teacher,course,...,4.0,3.0,4.0,1.0,1.0,3.0,6.0,5.0,6.0,6.0
GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,course,...,5.0,3.0,3.0,1.0,1.0,3.0,4.0,5.0,5.0,6.0
GP,F,15.0,U,LE3,T,1.0,1.0,at_home,other,other,...,4.0,3.0,2.0,2.0,3.0,3.0,10.0,NaN,8.0,10.0
GP,F,15.0,U,GT3,T,4.0,2.0,health,services,home,...,3.0,2.0,2.0,1.0,1.0,5.0,2.0,15.0,14.0,15.0
GP,F,NaN,U,GT3,T,3.0,3.0,other,other,home,...,4.0,3.0,2.0,1.0,2.0,5.0,4.0,6.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MS,M,20.0,U,LE3,A,2.0,2.0,services,services,course,...,5.0,5.0,4.0,4.0,5.0,4.0,11.0,9.0,9.0,9.0
MS,M,17.0,U,LE3,T,3.0,1.0,services,services,course,...,2.0,4.0,5.0,3.0,4.0,2.0,3.0,14.0,16.0,16.0
MS,M,21.0,R,GT3,T,1.0,1.0,other,other,course,...,5.0,5.0,3.0,3.0,3.0,3.0,3.0,10.0,8.0,7.0
